In [2]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import pandas as pd

from sklearn import linear_model, datasets

Build a regression model.

In [12]:
data = pd.read_csv('data_clean.csv')
data.rename(columns={'closest bike station id':'nearby_bike_station_id'}, inplace=True)
stations = pd.read_csv('tor_stations_sample.csv')
stations.head()

,empty_slots,free_bikes,id,latitude,longitude,name,timestamp
0,3,23,dcbcf9c681fbd3e77dfbea6df0e070bc,43.645857,-79.385365,Simcoe St / Wellington St W South,2023-11-17 15:55:45.502000-05:00
1,0,18,d1de2ddc0ea8a90d00ff717ad60a2f23,43.638497,-79.391920,Lower Spadina Ave / Lake Shore Blvd W,2023-11-17 15:55:44.622000-05:00
2,3,12,9fd381407b51a700753caec17607a127,43.647508,-79.386044,Simcoe St / King St W,2023-11-17 15:55:45.336000-05:00
3,3,16,f42363acf4af847ca6fcdbe9b48c8290,43.637981,-79.424146,Liberty St / Fraser Ave Green P,2023-11-17 15:55:45.472000-05:00
4,14,2,c4a01864ecb66e9b1ee3023d3205a281,43.680022,-79.340568,Langford Ave / Danforth Ave - SMART,2023-11-17 15:55:45.220000-05:00


In [20]:
merged_df = pd.merge(data, stations[['id', 'free_bikes']], left_on='nearby_bike_station_id', right_on='id')
merged_df.head()

,nearby_bike_station_id,poi_id,name,address,distance,longitude,latitude,categories,rating,price,...,rating_3,rating_4,rating_5,rating_6,rating_7,rating_8,rating_9,rating_10,id,free_bikes
0,dcbcf9c681fbd3e77dfbea6df0e070bc,r_BrIgzYcwo1NAuG9dLbpg,Pai Northern Thai Kitchen,18 Duncan Street,348.204192,-79.388720,43.647840,Thai,9.0,2,...,0,0,0,0,0,0,1,0,dcbcf9c681fbd3e77dfbea6df0e070bc,23
1,dcbcf9c681fbd3e77dfbea6df0e070bc,Yl2TN9c23ZGLUBSD9ks5Uw,Byblos,11 Duncan Street,290.701203,-79.388123,43.647546,"Mediterranean, Middle Eastern",9.0,4,...,0,0,0,0,0,0,1,0,dcbcf9c681fbd3e77dfbea6df0e070bc,23
2,dcbcf9c681fbd3e77dfbea6df0e070bc,jGO4T6rGX3EIOwOtg4bZqg,Lobster Burger Bar,214 King Street W,209.383312,-79.386768,43.647443,"Bars, Seafood, Burgers",8.0,2,...,0,0,0,0,0,1,0,0,dcbcf9c681fbd3e77dfbea6df0e070bc,23
3,dcbcf9c681fbd3e77dfbea6df0e070bc,ocLnCE2E29j-CoBQj1yaSA,Cafe Landwer - Adelaide & University,165 University Avenue,325.377024,-79.384910,43.648750,"Cafes, Breakfast & Brunch, Mediterranean",8.0,2,...,0,0,0,0,0,1,0,0,dcbcf9c681fbd3e77dfbea6df0e070bc,23
4,dcbcf9c681fbd3e77dfbea6df0e070bc,JMiaNitMzMbJm6Kh0RbT5A,Canoe,66 Wellington Street W,285.381498,-79.382159,43.646954,Canadian (New),8.0,4,...,0,0,0,0,0,1,0,0,dcbcf9c681fbd3e77dfbea6df0e070bc,23


In [24]:
merged_df.to_csv('merged_df.csv', index=False)

Provide model output and an interpretation of the results. 

In [32]:
X = merged_df.drop('free_bikes', axis=1)
y = merged_df['free_bikes']
X.columns

Index(['nearby_bike_station_id', 'poi_id', 'name', 'address', 'distance',
       'longitude', 'latitude', 'categories', 'rating', 'price',
       'numeric_station_id', 'price_1', 'price_2', 'price_3', 'price_4',
       'rating_2', 'rating_3', 'rating_4', 'rating_5', 'rating_6', 'rating_7',
       'rating_8', 'rating_9', 'rating_10'],
      dtype='object')

In [36]:
X = X[['distance', 'longitude', 'latitude', 'rating', 'price', 'numeric_station_id']]

In [30]:
y.head()

0    23
1    23
2    23
3    23
4    23
Name: free_bikes, dtype: int64

In [37]:
X = sm.add_constant(X)
lin_reg = sm.OLS(y, X)

In [38]:
model = lin_reg.fit()
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:             free_bikes   R-squared:                       0.081
Model:                            OLS   Adj. R-squared:                  0.079
Method:                 Least Squares   F-statistic:                     43.93
Date:                Sun, 19 Nov 2023   Prob (F-statistic):           1.07e-51
Time:                        20:04:01   Log-Likelihood:                -10039.
No. Observations:                2990   AIC:                         2.009e+04
Df Residuals:                    2983   BIC:                         2.013e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const               4378.2833    320

Removing lowest p-value variable, rerunning the model, and repeating that process until all p-values are below 0.05. 

In [39]:
X = X[['distance', 'longitude', 'latitude', 'rating', 'price']]

In [40]:
X = sm.add_constant(X)
lin_reg = sm.OLS(y, X)

In [41]:
model = lin_reg.fit()
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:             free_bikes   R-squared:                       0.081
Model:                            OLS   Adj. R-squared:                  0.079
Method:                 Least Squares   F-statistic:                     52.57
Date:                Sun, 19 Nov 2023   Prob (F-statistic):           1.99e-52
Time:                        20:05:12   Log-Likelihood:                -10039.
No. Observations:                2990   AIC:                         2.009e+04
Df Residuals:                    2984   BIC:                         2.013e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4390.4387    320.580     13.695      0.0

In [42]:
X = X[['longitude', 'latitude', 'rating', 'price']]

In [43]:
X = sm.add_constant(X)
lin_reg = sm.OLS(y, X)

In [44]:
model = lin_reg.fit()
print_model = model.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:             free_bikes   R-squared:                       0.080
Model:                            OLS   Adj. R-squared:                  0.079
Method:                 Least Squares   F-statistic:                     64.93
Date:                Sun, 19 Nov 2023   Prob (F-statistic):           1.00e-52
Time:                        20:06:01   Log-Likelihood:                -10041.
No. Observations:                2990   AIC:                         2.009e+04
Df Residuals:                    2985   BIC:                         2.012e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       4341.9555    319.448     13.592      0.0

The model was created by running a multivariate linear regression model first with all variables, and then removing the highest P>|t| values and rerunning the model, repeating that process until all P>|t| were below 0.05. 

Despite those steps above, our R-squared value is only 0.08, implying that we do not have a very good fit for our data.  

The largest impact on the dependent variable is the latitude, which is the angular distance from north or south of the equator. What this means in the context of the bikes stations, is that it indicates how far north or south the bike station is in Toronto. 

As Toronto is north of the equator, the latitude is always negative. With a negative coefficient, this implies that as you move north in toronto, the number of free bikes will *increase*. 

Looking back at the EDA completed, it could also be seen that the bik stations were focused around East-West corridors, or in other words were clustered around a similar latitudes. This clustering could be the reason that the latitude has the largest impact out of the independent variables. 

This also makes sense when thinking about the layout of Toronto. The city center is close to the water (ie. further south). The further south you are, the denser the city becomes. It is also more difficult to navigate the city via car in these dense neighbourhoods, which could make the city bikes more popular. 

# Stretch

How can you turn the regression model into a classification model?

Some classification models that could be computed may be the following:
- using the rating or price dummy variables to determine if there is a relationship between any independent variables and those ratings or prices
- group the number of free bikes into two bins, labelling the first bin as 1 and the second bin as 2. 
    - From here, we could try to predict which bin a station falls into based on the independent variables. 
- we may be able to also predict the rating or price based on the number of free bikes
- Using the location, and given the fact that we may have a relationship between the latitude and number of free bikes, we could try to determine if a station has bikes at all based on the latitude alone, or a combination of latitude and independent variables from the initial model